In [24]:
from statsbombpy import sb

import os

import pandas as pd

import warnings
warnings.filterwarnings("ignore")

email = "nathan.talbot@etu.uca.fr"
password = os.environ["mdp_statsbomb"]
creds = {"user" : email, "passwd" : password}

In [33]:
event = sb.events(3937511, creds = creds)
min(pd.DataFrame(event[event.type == "Shot"].location.tolist())[0])

90.0